In [ ]:
%load_ext autoreload
%autoreload 2

from pystorm.hal import HAL
from pystorm.PyDriver import bddriver as bd
from pystorm.hal.net_builder import NetBuilder

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

In [ ]:
np.random.seed(1)

# making a Y-by-X pool of D dims located at (LY, LX)

Y, X = (16, 16)
LY, LX = (16, 0)

N = X * Y
D = 1 # has to be 1 for this

SY = Y // 2
SX = X // 2

In [ ]:
hal = HAL()
net_builder = NetBuilder(hal)

# get the complete set of bad syn locations for the chip (slow PEs and high bias mismatch)
bad_syns, _ = net_builder.determine_bad_syns()

# create some nice taps, avoiding bad syns
syn_tap_matrix = NetBuilder.create_default_yx_taps(Y // 2, X // 2, D, 
                                                  bad_syn=bad_syns[LY // 2 : (LY + Y) // 2, 
                                                                   LX // 2 : (LX + X) // 2])
nrn_tap_matrix = NetBuilder.syn_taps_to_nrn_taps(syn_tap_matrix)
NetBuilder.make_taps_even(nrn_tap_matrix)

In [ ]:
# determine if syn_tap_matrix is right/left or up/down +1/-1 
# so I know how to cut it in two

plt.figure()
plt.imshow(syn_tap_matrix.reshape(SY, SX))
plt.title('corner coordinates\nCCW from top-left: (0, 0), (SY, 0), (SY, SX), (0, SX)')

print(syn_tap_matrix[0, 0])
print(syn_tap_matrix[SY-1, 0])
print(syn_tap_matrix[0, SX-1])
print(syn_tap_matrix[SY-1, SX-1])

# need diffusor cuts in terms of tile_ids
# cuts are on the north and west sides, left/right and top/bottom respectively
# we're making a vertical cut, so using west top/bottoms
# 4 syns to a tile

DIFFUSOR_NORTH_LEFT = bd.bdpars.DiffusorCutLocationId.NORTH_LEFT
DIFFUSOR_NORTH_RIGHT = bd.bdpars.DiffusorCutLocationId.NORTH_RIGHT
DIFFUSOR_WEST_TOP = bd.bdpars.DiffusorCutLocationId.WEST_TOP
DIFFUSOR_WEST_BOTTOM = bd.bdpars.DiffusorCutLocationId.WEST_BOTTOM

def slice_square_pool_in_half(hal):
    x_off = LX // 2 // 2
    y_off = LY // 2 // 2
    num_tiles_vert = SY // 2
    x_idx = SX // 2 // 2 #east of midline tile idx
    
    
    for y_idx in range(num_tiles_vert):
        #print(x_idx + x_off, y_idx + y_off)
        hal.driver.OpenDiffusorCutXY(0, x_idx + x_off, y_idx + y_off, DIFFUSOR_WEST_TOP)
        hal.driver.OpenDiffusorCutXY(0, x_idx + x_off, y_idx + y_off, DIFFUSOR_WEST_BOTTOM)


In [ ]:
# set up the Network object, do encoder/offset optimization
def run_network(diff_G, diff_R=1024, cut='none', biases=0, tap_matrix=nrn_tap_matrix):
    net = net_builder.create_single_pool_net(Y, X, loc_yx=(LY, LX), tap_matrix=tap_matrix, biases=biases, gain_divs=1)
    pool_obj = net.get_pools()[0]
    input_obj = net.get_inputs()[0]
    hal.map(net)
    
    hal.set_DAC_value('DAC_SOMA_REF', 1024)

    # make the diffusor broad
    hal.set_DAC_value('DAC_DIFF_G', diff_G)
    hal.set_DAC_value('DAC_DIFF_R', diff_R)

    # net is now mapped, try slicing the diffusor
    if cut == 'line':
        slice_square_pool_in_half(hal)
    elif cut == 'all':
        net_builder.open_all_diff_cuts()

    # determine fmax
    safe_fmaxes = net_builder.determine_safe_fmaxes()
    fmax = safe_fmaxes[pool_obj]

    # estimate encoders and offsets
    encoders, offsets, _, _ = net_builder.determine_encoders_and_offsets(pool_obj, input_obj, fmax, 
                                                                         num_sample_angles=3,
                                                                         solver='scipy_opt')
    
    return net, encoders, offsets

In [ ]:
# do exhaustive twiddle val search, like Terry does
# TODO come back and try with the calibration

import pickle
pck_fname = 'raw_offsets.pck'
REDO_SWEEP = False

if REDO_SWEEP:
    raw_offsets = np.zeros((7, N))
    for bias_idx, bias in enumerate([-3, -2, -1, 0, 1, 2, 3]):
        trick_net, trick_encoders, trick_offsets = run_network(diff_G=1024, diff_R=1024, cut='line', biases=bias)
        raw_offsets[bias_idx, :] = trick_offsets
    pickle.dump(raw_offsets, open(pck_fname, 'wb'))
else:
    raw_offsets = pickle.load(open(pck_fname, 'rb'))
    

In [ ]:
# make it like the soma_bias_twiddle calibration, relative to bias twiddle 0
all_offsets = raw_offsets.copy()
orig_offsets_at_3 = raw_offsets[6, :]
orig_offsets_at_0 = raw_offsets[3, :]
    
# subtract out 0 bias
for bias_idx, bias in enumerate([-3, -2, -1, 0, 1, 2, 3]):
    all_offsets[bias_idx, :] -= orig_offsets_at_0

In [ ]:
# plot resulting offsets
plt.figure()
plt.plot(raw_offsets)

plt.figure()
plt.plot(all_offsets)

# estimate slope of each neurons sampled twiddle levels
# the mismatch of each level is correlated
# this is a better way of estimating unsampled bias levels than what 
# the calibration does

def idx_to_mag(idx):
    return np.abs(idx - 3)

est_slope_p = np.zeros(N)
highest_sample_p = np.zeros(N, dtype=int)
est_slope_n = np.zeros(N)
highest_sample_n = np.zeros(N, dtype=int)
for n in range(N):
    nrn_offsets = all_offsets[:, n]
    valid = ~np.isnan(nrn_offsets)
    valid_off = nrn_offsets.copy()
    valid_off[~valid] = 0
    if len(valid_off) > 0:
        biggest = np.max(valid_off)
        biggest_arg = np.argmax(valid_off)
        if biggest > 0 and biggest_arg > 3:
            est_slope_p[n] = biggest / idx_to_mag(biggest_arg)
            highest_sample_p[n] = idx_to_mag(biggest_arg)
            
        smallest = np.min(valid_off)
        smallest_arg = np.argmin(valid_off)
        if smallest < 0 and smallest_arg < 3:
            est_slope_n[n] = smallest / idx_to_mag(smallest_arg)
            highest_sample_n[n] = idx_to_mag(smallest_arg)
            
# compute global mean offsets
means = []
for bias_level in range(7):
    bias_offsets = all_offsets[bias_level, :]
    means.append(np.mean(bias_offsets[~np.isnan(bias_offsets)]))
    
# weight estimated slopes and global means together to fill in data
all_offsets_est = all_offsets.copy()
for n in range(N):
    for bias_level in range(7):
        if np.isnan(all_offsets_est[bias_level, n]):
            assign = False
            if bias_level > 3:
                highest_sample = highest_sample_p[n]
                est_slope = est_slope_p[n]
                if idx_to_mag(bias_level) > highest_sample:
                    assign = True
            elif bias_level < 3:
                highest_sample = highest_sample_n[n]
                est_slope = est_slope_n[n]
                if idx_to_mag(bias_level) > highest_sample:
                    assign = True
            if assign:
                nrn_est_offset = idx_to_mag(bias_level) * est_slope
                all_offsets_est[bias_level, n] = nrn_est_offset * highest_sample / 3 + \
                                                 means[bias_level] * (1 - highest_sample / 3)

plt.figure()
plt.plot(all_offsets_est)
print('hi')


In [ ]:
def optimize_twiddles_once(cut, diff_G, diff_R, tap_matrix=None):

    def run_curr_network(biases):
        if tap_matrix is None:
            return run_network(diff_G, diff_R=diff_R, cut=cut, biases=biases)
        else:
            return run_network(diff_G, diff_R=diff_R, cut=cut, biases=biases, tap_matrix=tap_matrix)

    _, encoders, offsets = run_curr_network(3)
    offsets_at_3 = offsets

    bias_settings, new_offsets, good, bin_counts, dbg = \
        NetBuilder.pick_good_twiddles(encoders, offsets_at_3, all_offsets_est, policy='greedy_flat')

    fs = (15, 15)
    xylim = (0, 800, -800, 1500)
    NetBuilder.plot_neuron_yield_cone(encoders, new_offsets, good,
                                     (encoders, offsets, bias_settings),
                                      title='expected',
                                      figsize=fs,
                                      xylim=xylim)

    net_opt, encoders_opt, offsets_opt = run_curr_network(bias_settings)

    NetBuilder.plot_neuron_yield_cone(encoders_opt, offsets_opt, good,
                                     (encoders, offsets, bias_settings),
                                      title='trick opt vs trick',
                                      figsize=fs,
                                      xylim=xylim)

    good_orig = np.sum(NetBuilder.get_good_mask(encoders, offsets))
    good_exp = np.sum(NetBuilder.get_good_mask(encoders, new_offsets))
    good_ver = np.sum(NetBuilder.get_good_mask(encoders_opt, offsets_opt))
    print('good orig:', good_orig)
    print('good exp:', good_exp)
    print('good ver:', good_ver)
    
    return encoders_opt, offsets_opt, good_ver, net_opt
        
def do_validation_exp(encoders_opt, offsets_opt, net_opt):
    NUM_VAL_SAMPLES = 20
    val_pts = np.linspace(-1, 1, NUM_VAL_SAMPLES).reshape((NUM_VAL_SAMPLES, 1))

    pool_obj = net_opt.get_pools()[0]
    inp_obj = net_opt.get_inputs()[0]
    safe_fmaxes = net_builder.determine_safe_fmaxes()
    fmax = safe_fmaxes[pool_obj]
    rmse, meas_A, est_A = net_builder.validate_est_encs(encoders_opt, offsets_opt, 
                                                        pool_obj, inp_obj, val_pts, fmax)
    return val_pts, meas_A

def plot_validation_exp(encoders_opt, offsets_opt, val_pts, meas_A, lines_per_plot=None):

    clean_encs = encoders_opt.copy()
    unest = np.isnan(offsets_opt)
    clean_encs[unest, :] = 0
    clean_offsets = offsets_opt.copy()
    clean_offsets[unest] = 0

    est_A = np.maximum(0, np.dot(val_pts, clean_encs.T) + clean_offsets)

    if lines_per_plot is not None:
        n_neurons = meas_A.shape[1]
        plot_nrn_idxs = np.random.permutation(np.arange(n_neurons))[:lines_per_plot]
        print(plot_nrn_idxs)
        meas_A_plot = meas_A[:, plot_nrn_idxs]
        est_A_plot = est_A[:, plot_nrn_idxs]
    else:
        meas_A_plot = meas_A
        est_A_plot = est_A
        
    plt.figure(figsize=(15,15))
    plt.gca().set_prop_cycle(None)
    plt.plot(val_pts, meas_A_plot, '.-')
    plt.gca().set_prop_cycle(None)
    plt.plot(val_pts, est_A_plot, '-')
    plt.axis([-1, 1, 0, 500]) 
    plt.title('estimated vs measured tuning curves')
    
    plt.figure(figsize=(15,15))
    plt.plot(val_pts, est_A)
    plt.axis([-1, 1, 0, 500]) 
    plt.title('estimated tuning curves')

    plt.figure()
    is_est = ~unest
    gains = NetBuilder.get_gains(encoders_opt[is_est, :])
    intercepts = -offsets_opt[is_est] / gains
    plt.hist(intercepts, bins=np.linspace(-2, 2, 21))
    plt.title('intercept distribution')
        

In [ ]:
encs, offs, good_ct, net = optimize_twiddles_once('line', 1024, 1024)
val_pts, meas_A = do_validation_exp(encs, offs, net)

In [ ]:
plot_validation_exp(encs, offs, val_pts, meas_A, lines_per_plot=50)

In [ ]:
# sweep diffusor spread, measuring for yield
def run_diffusor_sweep(cut, diff_Gs, diff_Rs, tap_matrix=None):
    encs = []
    offs = []
    good_cts = []
    for diff_G, diff_R in zip(diff_Gs, diff_Rs):
        optimize_twiddles_once(cut, diff_G, diff_R, tap_matrix)
        
        encs.append(encoders_opt)
        offs.append(encoders_opt)
        good_cts.append(good_ver)
        
    return encs, offs, good_cts, net_opt
        

In [ ]:
diff_Rs = [500, 1024, 1024]
diff_Gs = [1024, 1024, 500]
cut = 'line'
run_diffusor_sweep(cut, diff_Gs, diff_Rs, plot_est_encs=True)


In [ ]:
diff_Rs = [1024, 500, 100]
diff_Gs = [1024]*len(diff_Rs)
cut = 'none'
run_diffusor_sweep(cut, diff_Gs, diff_Rs)

In [ ]:
# see how much bad_syns helps

default_syn_tap_matrix = NetBuilder.create_default_yx_taps(Y // 2, X // 2, D)
default_nrn_tap_matrix = NetBuilder.syn_taps_to_nrn_taps(default_syn_tap_matrix)
NetBuilder.make_taps_even(default_nrn_tap_matrix)

diff_Rs = [1024]
diff_Gs = [1024]

cut = 'line'
run_diffusor_sweep(cut, diff_Gs, diff_Rs, tap_matrix=default_nrn_tap_matrix)